k-최근접 이웃의 다중 분류

In [6]:
import pandas as pd
import numpy as np

fish = pd.read_csv("https://bit.ly/fish_csv_data")
print(fish.head())

fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fish_target = fish[['Species']].to_numpy()


  Species  Weight  Length  Diagonal   Height   Width
0   Bream   242.0    25.4      30.0  11.5200  4.0200
1   Bream   290.0    26.3      31.2  12.4800  4.3056
2   Bream   340.0    26.5      31.1  12.3778  4.6961
3   Bream   363.0    29.0      33.5  12.7300  4.4555
4   Bream   430.0    29.0      34.0  12.4440  5.1340


In [7]:
# target의 고유값들 확인
print(np.unique(fish_target))

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
    train_test_split(fish_input,fish_target,\
        stratify=fish_target,random_state=42)
    
# train_input = train_input.reshape(-1,1)
# test_input = test_input.reshape(-1,1)

# 다항 특성 만들기
# poly = PolynomialFeatures(degree=5,include_bias=False)
# poly.fit(train_input)

# train_poly = poly.transform(train_input)
# test_poly = poly.transform(test_input)

# 스케일링
std = StandardScaler()
std.fit(train_input)

train_scaled = std.transform(train_input)
test_scaled = std.transform(test_input)



In [12]:
from sklearn.neighbors import KNeighborsClassifier
from numpy import ravel

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(fish_input,ravel(fish_target))

print(kn.classes_)

print(kn.predict(test_scaled[:5]))

proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))


['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
['Smelt' 'Smelt' 'Smelt' 'Smelt' 'Smelt']
[[0.     0.     0.3333 0.     0.     0.6667 0.    ]
 [0.     0.     0.3333 0.     0.     0.6667 0.    ]
 [0.     0.     0.3333 0.     0.     0.6667 0.    ]
 [0.     0.     0.3333 0.     0.     0.6667 0.    ]
 [0.     0.     0.3333 0.     0.     0.6667 0.    ]]


# 로지스틱 회귀(이진 분류)

In [10]:
# -> 시그모이드 함수(tanh) 
# 0.5보다 크면 양성 클래스
# 0.5보다 작으면 음성 클래스

In [11]:
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

print(lr.predict(train_bream_smelt[:5]))

print(lr.predict_proba(train_bream_smelt[:5]))


IndexError: boolean index did not match indexed array along dimension 1; dimension is 5 but corresponding boolean dimension is 1

In [ ]:
print(train_scaled.shape)     # (595, ?) 혹은 (119, ?)
print(train_target.shape)     # (119,)


(595, 1)
(119, 1)
